In [1]:
import pandas as pd, numpy as np, matplotlib as mpl, xlrd, string
from fuzzywuzzy import fuzz,process

/home/rstudio/.local/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Add continent columns 

In [2]:
#Read and clean raw data and country/continent lookup table
RawData = pd.read_excel("RefugeeData.xlsx")
df = RawData.drop(axis = 1, labels=['Population type'])
ContinentLookup = pd.read_excel('ContinentsList.xlsx')
ContinentLookup = ContinentLookup.rename(columns={'Country or territory of asylum or residence':'Country'})
df = df.rename(columns={'Country / territory of asylum/residence': 'CountryTo','Origin': 'CountryFrom'})

In [3]:
#add ContinentTo to df by joining with continent lookup table
df = df.merge(ContinentLookup, left_on = "CountryTo", right_on = "Country", how = "left", copy=False).drop(labels = ["Country"], axis = 1)
df = df.rename(columns={'Continent': 'ContinentTo'})

In [4]:
#add ContinentFrom to df by joining with continent lookup table
df = df.merge(ContinentLookup, left_on = "CountryFrom", right_on = "Country").drop(labels = ["Country"], axis = 1)
df = df.rename(columns={'Continent': 'ContinentFrom'})

In [5]:
#Remove rows with all various locations
VariousString = "Various/Unknown" + "Various/Unknown" + "Various" + "Various"
AllColumns = df.CountryTo + df.CountryFrom + df.ContinentFrom + df.ContinentTo
df["RemoveVarious"] = AllColumns
df = df.loc[df.RemoveVarious != VariousString]
df = df.drop(columns="RemoveVarious")

In [6]:
#Change rows that have North Africa to Africa
df['ContinentFromClean'] = np.where(
                                     df.ContinentFrom=="North Africa"
                                    ,"Africa"
                                    ,df.ContinentFrom
                                    )

df['ContinentToClean'] = np.where(
                                    df.ContinentTo=="North Africa"
                                   ,"Africa"
                                   ,df.ContinentTo
                                  )


In [7]:
#Drop old columns
df = df.drop('ContinentFrom', 1)
df = df.drop('ContinentTo', 1)
df = df.rename(columns={'ContinentFromClean': 'ContinentFrom','ContinentToClean': 'ContinentTo'})
#Reorder columns
df = df[['Year','Value','CountryFrom','CountryTo','ContinentFrom','ContinentTo']]
#Remove rows where value = '*'
df = df.loc[df.Value != '*']
#Export final table to csv
df.to_csv("CleanRefugeeTable.csv")

# Create country lookup table from countries in refugee & data 

In [8]:
CountryPopulationData = pd.read_excel('CountryPopulations.xlsx')
Continents = pd.read_excel('ContinentsList.xlsx')

# Make list of unique countrys to lookup
CountryList = pd.concat([CountryPopulationData['Country Name']
        ,Continents['Country or territory of asylum or residence']]).drop_duplicates().reset_index().drop(columns='index')
CountryList.columns = ['Country']

In [9]:
#make dataframe with fuzzy matching loop to pull best matches as column

CleanNames = pd.DataFrame({"Country":[],
        "BestScore":[],
        "BestScoreCountry":[]})

for i, j in enumerate(CountryList.Country):
    
    MatchPrevious = CountryList.loc[i].Country
    ScoreDF = pd.DataFrame(columns=["Country","Score"])
    print(i)
    if i > 0:
        for row in CountryList.Country[:i]:
            temp = pd.DataFrame({
                               "Country":[row]
                              ,"Score":[fuzz.token_set_ratio(MatchPrevious,row)]
                              })
            ScoreDF = ScoreDF.append(temp)
      
        BestScore = pd.DataFrame({"Country":[MatchPrevious],
        "BestScore":[max(ScoreDF.Score)],
        "BestScoreCountry":[np.array(ScoreDF.loc[ScoreDF.Score == max(ScoreDF.Score),"Country"])[0]]})
      
  
        CleanNames = CleanNames.append(BestScore)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
#Add column and run loop to retrieve country strings from best matchs

CleanNames['Clean'] = ""
CleanNames = CleanNames.reset_index().drop(columns='index')

for i, j in enumerate(CleanNames["Country"]):
    if CleanNames.loc[i,"BestScore"] > 90:
        CleanNames.loc[i, "Clean"] = CleanNames.loc[i, "BestScoreCountry"]
    else:
        CleanNames.loc[i,"Clean"] = CleanNames.loc[i,"Country"]
        
CleanNames.to_csv('CountryFuzzy.csv')

In [11]:
#Wrangle country lookup table
CountryLookup = pd.read_excel('CountryLookup.xlsx')
CountryLookup = CountryLookup.drop(columns=['Unnamed: 0','BestScore'])
CountryLookup = CountryLookup.loc[CountryLookup.Cleaner != 'remove']
CountryLookup = CountryLookup.reset_index().drop(columns='index')
CountryPopulationData = CountryPopulationData.drop(["Country Code","Indicator Name","Indicator Code"], axis=1)

In [12]:
#Wrangle and merge population table with the lookup for a clean name column
Populations = pd.melt(CountryPopulationData, 'Country Name', var_name='Date', value_name='Total Population')
Populations = Populations.merge(CountryLookup,how='left',left_on='Country Name',right_on='Country')
Populations = Populations.drop(columns='Country')
Populations.to_csv("Populations.csv")

# Add population column to df

In [13]:
df = pd.read_csv("CleanRefugeeTable.csv")
Pop = pd.read_csv("Populations.csv")
CountryLookup = pd.read_excel("CountryLookup.xlsx")
len(df)

100422

In [14]:
#Clean all dataframes for merge
Pop = Pop.drop(columns=["Cleaner"])
df = df.loc[df.CountryTo.isna()==False]
df = df.drop(columns='Unnamed: 0')
Pop = Pop.drop(columns='Unnamed: 0')
CountryLookup = CountryLookup.drop(columns=["Unnamed: 0"])
len(df)

100421

In [15]:
#Clean country column

Pop = Pop.merge(CountryLookup,how='left',left_on='Country Name', right_on='Country')
Pop = Pop.drop(columns=["Country","BestScore"])

#add cleancountry column to Pop df to remove na
Pop["CleanCountry"] = np.where(pd.isna(Pop["Cleaner"]),
                                           Pop["Country Name"],
                                         Pop["Cleaner"])

Pop = Pop.drop(columns='Cleaner')

In [16]:
#merge main df and countrylookup to get the clean country column
df = df.merge(CountryLookup, how='left',left_on='CountryFrom',right_on='Country')
df = df.drop(columns=["BestScore","Country"])
Pop.columns = ['Country','Year','Population',"CleanCountry"]

In [17]:
#merge df and Pop to add yearly populations to country data 
CleanTable = pd.merge(df, Pop,  how='left', left_on=['Year','Cleaner'], right_on = ['Year','CleanCountry'])
CleanTable = CleanTable.drop(columns=["Country","CleanCountry","Cleaner"])
CleanTable = CleanTable.rename(columns={'Population': 'CountryFromPopulation'})
len(CleanTable)

100421

In [18]:
# Percentage of N/A rows in the Population column
print(len(CleanTable.loc[CleanTable.CountryFromPopulation.isna()==True])/len(CleanTable))

0.06153095468079386


In [19]:
CleanTable.to_csv("CleanedRefugeeData.csv")